# Homework 1
### COSC 594 Bioinformatics
### Started 01/15/18, Due 01/23/18 
### Elliot Greenlee

1\. On my honor, I agree to the terms of the Academic Code of Honor.

2\. See included file, lambda.fasta

In [1]:
# Input a genome from a file in fasta format
def input_genome(filename, chosen_gen):    
    with open(filename, 'r') as f:
        header = f.readline()
        current_gen = 1
        genome = ""
        for line in f:
            if line[0] is '>':
                if current_gen is chosen_gen:
                    genome = genome.replace('\n', '')
                    genome = genome.replace('N', 'A')
                    return header, genome
                else:
                    header = line
                    current_gen += 1
            else:
                genome = genome + line
          
        if current_gen is chosen_gen:
            genome = genome.replace('\n', '')
            genome = genome.replace('N', 'A')
            return header, genome
        else:
            print("This file only has {} genomes".format(current_gen))
            exit(1)

# Output a genome in fasta format
def output_genome(filename, header, genome):
    with open(filename, 'w') as f:
        f.write(header)
        f.write(genome)
        f.write("\n")

In [2]:
# 3. Reverse complement the lambda genome. The results can be found in lambda.rev.fasta
# Reverse a genome
def reverse(genome):
    rev_genome = genome[::-1]
    return rev_genome

# Complement a genome
def complement(genome):
    tmp_genome = genome.replace('A', '1')
    tmp_genome = tmp_genome.replace('T', '2')
    tmp_genome = tmp_genome.replace('C', '3')
    tmp_genome = tmp_genome.replace('G', '4')
    
    comp_genome = tmp_genome.replace('1', 'T')
    comp_genome = comp_genome.replace('2', 'A')
    comp_genome = comp_genome.replace('3', 'G')
    comp_genome = comp_genome.replace('4', 'C')
    
    return comp_genome

# Input the lambda genome
header, genome = input_genome("lambda.fasta", 1)

# Reverse complement the lambda genome
rev_comp_genome = reverse(complement(genome))

# Output the reverse complement of the lambda genome
output_genome("lambda.rev.fasta", "> reversed", rev_comp_genome)

In [3]:
# 4. Report the nucleotide frequencies and the dinucleotide frequencies of lambda
import numpy as np
import regex as re

nucleotides = ['A', 'T', 'C', 'G']

# Compute nucleotide frequencies for a genome
def freq_nucleotides(genome):
    length = len(genome) * 1.0
    frequencies = {}
    for nucleotide in nucleotides:
        frequencies[nucleotide] = genome.count(nucleotide) / length
    
    return frequencies

# Compute dinucleotide frequencies for a genome
def freq_dinucleotides(genome):
    num_dinucleotides = len(genome) - 1.0
    
    frequencies = {}
    # Iterate over dinucleotides
    for nucleotide1 in nucleotides:
        for nucleotide2 in nucleotides:
            dinucleotide = nucleotide1 + nucleotide2
            frequencies[dinucleotide] = len(re.findall(dinucleotide, genome, overlapped=True)) / num_dinucleotides
    
    return frequencies

# Input the lambda genome
header, genome = input_genome("lambda.fasta", 1)

# Compute the nucleotide frequencies of the lambda genome
print("Nucleotide Frequencies")
print(freq_nucleotides(genome))
print()

# Compute the dinucleotide frequencies
print("Dinucleotide Frequencies")
frequencies = freq_dinucleotides(genome)
for dinucleotide in frequencies:
    print(dinucleotide, frequencies[dinucleotide])

Nucleotide Frequencies
{'A': 0.2542987918023999, 'T': 0.24712382994515691, 'C': 0.23425838109768668, 'G': 0.2643189971547565}

Dinucleotide Frequencies
AA 0.07612214181150904
AT 0.06880270509886394
AC 0.053050452567988286
AG 0.056328735489989894
TA 0.04474134553926723
TT 0.06896765015154327
TC 0.05519473825281953
TG 0.07822519123317045
CA 0.06630791117708913
CT 0.05228758169934641
CC 0.05148347456753469
CG 0.06418424362384281
GA 0.06713263644048577
GT 0.057070988227046864
GC 0.07453454567947053
GG 0.06556565844003216


5\. See included files, human_mito.fasta and neander_sample.fasta

In [4]:
# 6. Compute the log of the probability of the Neanderthal sequence 
# for a multinomial model and an order 3 markov model, both trained on 
# the human mitochondrial genome
import math
import numpy as np

nucleotides = ['A', 'T', 'C', 'G']

def log_multinomial_prob(training_genome, testing_genome):
    # Calculate the nucleotide frequencies from training
    nucleotide_frequencies = freq_nucleotides(training_genome)
    
    # Calculate testing sample log probability
    total_probability = 0
    for nucleotide in nucleotides:
        total_probability += testing_genome.count(nucleotide) * math.log(nucleotide_frequencies[nucleotide])
                
    return total_probability

def log_markov_prob(training_genome, testing_genome):
    order = 3
    
    # Calculate the nucleotide frequencies from training
    nucleotide_frequencies = freq_nucleotides(training_genome)
    
    # Create dictionary of the transitions for all possible 4 letter combinations
    transitions = {}
    for nucleotide1 in nucleotides:
        for nucleotide2 in nucleotides:
            for nucleotide3 in nucleotides:
                t_sum = 0.0
                tmp = {}
                for nucleotide4 in nucleotides:
                    seq = nucleotide1 + nucleotide2 + nucleotide3 + nucleotide4
                    # Find all occurences of this 4 nucleotide sequence
                    count = len(re.findall(seq, training_genome, overlapped=True))
                    tmp[seq] = count
            
                    t_sum += count
                
                # Find transition value by dividing by sum of transitions for this final letter
                for key in tmp:
                    transitions[key] = tmp[key] / t_sum
    
    # Optionally print all transition values
    #for key in transitions:
        #print(key, transitions[key])
        
    counts = {}
    for nucleotide1 in nucleotides:
        for nucleotide2 in nucleotides:
            for nucleotide3 in nucleotides:
                for nucleotide4 in nucleotides:
                    seq = nucleotide1 + nucleotide2 + nucleotide3 + nucleotide4
                    counts[seq] = len(re.findall(seq, testing_genome, overlapped=True))
                    
                    
    total_probability = 0
    
    for nucleotide in testing_genome[0:order]:
        total_probability += math.log(nucleotide_frequencies[nucleotide])
        
    for seq in transitions:
        total_probability += counts[seq] * math.log(transitions[seq])
        
    return total_probability

# Input the human mitochondrial genome for training and the neanderthal genome for testing
training_header, training_genome = input_genome("human_mito.fasta", 1)
testing_header, testing_genome = input_genome("neander_sample.fasta", 1)

# Calculate the log of the multinomial probability
print("Multinomial Probability")
print(log_multinomial_prob(training_genome, testing_genome))
print()

# Calculate the log of the order 3 Markov probability
print("Order 3 Markov Probability")
print(log_markov_prob(training_genome, testing_genome))

Multinomial Probability
-452.6623122326945

Order 3 Markov Probability
-451.7966057821607


In [5]:
# 7. Generate a random sequence of 20,000 characters using a Markov model 
# of order 3 trained on the human mitochondrial genome. 
# See random.fasta for the genome

import random

def generate_random_sequence(training_genome, length):
    order = 3
    
    # Calculate the nucleotide frequencies from training
    nucleotide_frequencies = freq_nucleotides(training_genome)
    
    # Create dictionary of the transitions for all possible 4 letter combinations
    transitions = {}
    for nucleotide1 in nucleotides:
        for nucleotide2 in nucleotides:
            for nucleotide3 in nucleotides:
                t_sum = 0.0
                tmp = {}
                for nucleotide4 in nucleotides:
                    seq = nucleotide1 + nucleotide2 + nucleotide3 + nucleotide4
                    # Find all occurences of this 4 nucleotide sequence
                    count = len(re.findall(seq, training_genome, overlapped=True))
                    tmp[seq] = count
            
                    t_sum += count
                
                # Find transition value by dividing by sum of transitions for this final letter
                for key in tmp:
                    transitions[key] = tmp[key] / t_sum
                    
    random_genome = ""
    
    border = {}
    border['A'] = nucleotide_frequencies['A']
    border['T'] = border['A'] + nucleotide_frequencies['T']
    border['C'] = border['T'] + nucleotide_frequencies['C']
    border['G'] = border['C'] + nucleotide_frequencies['G']
    
    for i in range(0, order):
        rand = random.random()
        if rand <= border['A']:
            random_genome += 'A'
        elif rand > border['A'] and rand < border['T']:
            random_genome += 'T'
        elif rand > border['T'] and rand < border['C']:
            random_genome += 'C'
        else:
            random_genome += 'G'
            
    for i in range(0, length-order):
        current_seq = random_genome[i:i+order]
        border = {}
        border['A'] = transitions[current_seq + 'A']
        border['T'] = border['A'] + transitions[current_seq + 'T']
        border['C'] = border['T'] + transitions[current_seq + 'C']
        border['G'] = border['C'] + transitions[current_seq + 'G']
        
        rand = random.random()
        
        if rand <= border['A']:
            random_genome += 'A'
        elif rand > border['A'] and rand < border['T']:
            random_genome += 'T'
        elif rand > border['T'] and rand < border['C']:
            random_genome += 'C'
        else:
            random_genome += 'G'
        
    return random_genome

# Input the human mitochondrial genome for training
training_header, training_genome = input_genome("human_mito.fasta", 1)

# Generate a random 20,000 nucleotide sequence
random_genome = generate_random_sequence(training_genome, 20000)

# Output the randome sequence genome
output_genome("random.fasta", "> randomly generated using order 3 Markov model trained on the human mitochondrial genome", random_genome)